In [1]:
from collections.abc import Callable
import crcmod

In [2]:
crc8 = crcmod.mkCrcFun(0x107, rev=False, initCrc=0x00, xorOut=0x00)
crc16 = crcmod.mkCrcFun(0x18005, rev=False, initCrc=0xffff, xorOut=0x0000)
crc32 = crcmod.mkCrcFun(poly=0x104c11db7, rev=True, initCrc=0x00000000, xorOut=0xffffffff)

In [3]:
datagrams = [
    b"\x00\x01\x1A\x04\x00\x1F\x22\x04\x26\x80",
    b"\x00\x01\x1A\x04\x00\x1F\x29\xE9\xC0\x77",
    b"\x00\x01\x1A\x04\x00\x1F\x52\xA8\xFA\x18",
    b"\x00\x01\x1A\x04\x00\x1F\x6B\x5F\x34\x13",
]

In [4]:
def checksum(payload: bytes, method: Callable = crc32, size: int = 4, order: str = "big") -> bytes:
    return method(payload).to_bytes(size, order)

In [5]:
for method, size in [(crc8, 1), (crc16, 2), (crc32, 4)]:
    checksums = [checksum(datagram[:-size], method=method, size=size) for datagram in datagrams]
    trials = [datagram[-size:] for datagram in datagrams]
    checked = checksum == trials
    print(f"{checked}: {trials} {checksums}")

False: [b'\x80', b'w', b'\x18', b'\x13'] [b'\xb1', b'K', b'(', b'\xe8']
False: [b'&\x80', b'\xc0w', b'\xfa\x18', b'4\x13'] [b'\x7f\xac', b'\xc7\xc1', b'\\B', b'Hs']
False: [b'"\x04&\x80', b')\xe9\xc0w', b'R\xa8\xfa\x18', b'k_4\x13'] [b'9\x07\x9a\xc1', b'9\x07\x9a\xc1', b'9\x07\x9a\xc1', b'9\x07\x9a\xc1']
